Формат файла: login tuid docid jud cjud.

Пояснение к формату:  
login — логин асессора;  
uid — id асессора (user id); docid — id оцениваемого документа (document id);  
jud — оценка асессора (judgement);  
cjud — правильная оценка (correct judgement);  
разделитель — табуляция \t.

Оценки могут принимать значение [0, 1], т.е. задание, которое сделали асессоры, имеет бинарную шкалу.

Используя данные об оценках, установите, какие асессоры хуже всего справились с заданием. На какие показатели вы ориентировались и какие метрики вы использовали для ответа на этот вопрос? Можно ли предложить какие-то новые метрики для подсчета качества асессоров с учетом природы оценок у этого бинарного задания?

Опишите подробно все этапы вашего решения.

In [1]:
import pandas as pd

#from sklearn.metrics import confusion_matrix

In [25]:
df = pd.read_csv('data_task3.csv', delimiter= '\t')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   login   250000 non-null  object
 1   uid     250000 non-null  int64 
 2   docid   250000 non-null  int64 
 3   jud     250000 non-null  int64 
 4   cjud    250000 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 9.5+ MB


In [27]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
login,250000,600,assessor191,484,NaN,NaN,NaN,NaN,NaN,NaN,NaN
uid,250000,NaN,NaN,NaN,299.326,173.132,0,150,299,449,599
docid,250000,NaN,NaN,NaN,24999.5,14433.8,0,12499.8,24999.5,37499.2,49999
jud,250000,NaN,NaN,NaN,0.23812,0.425934,0,0,0,0,1
cjud,250000,NaN,NaN,NaN,0.11992,0.324869,0,0,0,0,1


Здесь мы сталкиваемся с обычной задачей классиифкации, будем строить **confusion matrix** и считать метрики для всех ассесоров.  Как обычно обозначим виды решений:  
TP - True Positive  - столбцы jud=cjud=1  
FN - False Negative - jud=0 cjud=1  
FP - False Positive - jud=1 cjud=0  
TN - True Negative - jud=0 chud=0  
Закодируем в новом столбце итог для удобства

In [39]:
df.loc[(df.jud==df.cjud) & (df.jud==1), 'pred'] = 'TP'
df.loc[(df.jud==df.cjud) & (df.jud==0), 'pred'] = 'TN'
df.loc[(df.jud!=df.cjud) & (df.jud==0), 'pred'] = 'FN'
df.loc[(df.jud!=df.cjud) & (df.jud==1), 'pred'] = 'FP'

In [42]:
df.sample(2)

,login,uid,docid,jud,cjud,pred
121363,assessor589,589,24272,1,0,FP
122284,assessor288,288,24456,0,0,TN


Группируем данные по ассессорам

In [57]:
assessors = df.groupby(['login', 'pred'])['pred'].count().unstack()
assessors

pred,FN,FP,TN,TP
login,,,,
assessor0,6,59,287,49
assessor1,4,78,289,41
assessor10,13,79,321,30
assessor100,15,68,299,35
assessor101,9,41,320,33
...,...,...,...,...
assessor95,10,29,349,40
assessor96,19,69,324,39
assessor97,7,33,346,37



Создадим метрики, можно было использовать sklearn, 
но для большого числа ассесоров проще написать вручную сразу в датафрейме

Для начала создадим основные метрики **accuracy, precision, recall и рассчитаем гармоничную метрику f1**

In [63]:
assessors['accuracy'] = (assessors['TP'] + assessors['TN']) / (
    assessors['TP'] + assessors['TN'] + assessors['FP'] + assessors['FN'])

assessors['recall'] = assessors['TP'] / (assessors['TP'] + assessors['FN'])

assessors['specifity'] = assessors['TN'] / (assessors['TN'] + assessors['FP'])

assessors['precision'] = assessors['TP'] / (assessors['TP'] + assessors['FP'])

assessors['f1_score'] = 2* assessors['precision'] * assessors['recall'] / (assessors['precision'] + 
                                                                          assessors['recall'])

In [64]:
assessors

pred,FN,FP,TN,TP,accuracy,recall,specifity,precision,f1_score
login,,,,,,,,,
assessor0,6,59,287,49,0.837905,0.890909,0.829480,0.453704,0.601227
assessor1,4,78,289,41,0.800971,0.911111,0.787466,0.344538,0.500000
assessor10,13,79,321,30,0.792325,0.697674,0.802500,0.275229,0.394737
assessor100,15,68,299,35,0.800959,0.700000,0.814714,0.339806,0.457516
assessor101,9,41,320,33,0.875931,0.785714,0.886427,0.445946,0.568966
...,...,...,...,...,...,...,...,...,...
assessor95,10,29,349,40,0.908879,0.800000,0.923280,0.579710,0.672269
assessor96,19,69,324,39,0.804878,0.672414,0.824427,0.361111,0.469880
assessor97,7,33,346,37,0.905437,0.840909,0.912929,0.528571,0.649123


In [105]:
assessors.f1_score.sort_values(ascending=False).tail(20)

login
assessor92     0.385093
assessor201    0.384615
assessor13     0.384106
assessor22     0.377622
assessor235    0.377358
assessor264    0.375839
assessor290    0.371429
assessor430    0.370370
assessor398    0.364964
assessor276    0.363636
assessor460    0.347826
assessor335    0.341463
assessor354    0.333333
assessor111    0.283186
assessor234    0.238806
assessor390    0.207407
assessor118    0.189723
assessor3      0.178571
assessor56     0.163121
assessor163    0.103896
Name: f1_score, dtype: float64

Отсортируем по f1_score и посмотрим, что находится внизу списка

In [68]:
assessors[assessors.f1_score<0.2]

pred,FN,FP,TN,TP,accuracy,recall,specifity,precision,f1_score
login,,,,,,,,,
assessor118,15,190,162,24,0.475703,0.615385,0.460227,0.112150,0.189723
assessor163,34,35,326,4,0.827068,0.105263,0.903047,0.102564,0.103896
assessor3,24,206,171,25,0.460094,0.510204,0.453581,0.108225,0.178571
assessor56,23,213,152,23,0.425791,0.500000,0.416438,0.097458,0.163121


Рассмотрим некоторых ассессоров.  У всех очень мало верно-положительных оценок, дополнительно:   
118, 3, 56 - очень много ложноположительных оценок  
163 - всего 4 правильне (TP) оценки, хорошая оценка *accuracy* не должна вводить в заблуждение, т.к. большой вклад вносят правильно "указанные" истинно-ложные оценки.

Но вообще стоит тоже самое сделать для самих документов, возмодно есть такие в которых ошибаются абсолютно все.

In [72]:
# группируем документы
doc_ids = df.groupby(['docid', 'pred'])['pred'].count().unstack()
doc_ids

pred,FN,FP,TN,TP
docid,,,,
0,NaN,NaN,5.0,NaN
1,1.0,NaN,NaN,4.0
2,NaN,1.0,4.0,NaN
3,NaN,1.0,4.0,NaN
4,NaN,1.0,4.0,NaN
...,...,...,...,...
49995,NaN,NaN,5.0,NaN
49996,NaN,NaN,5.0,NaN
49997,NaN,1.0,4.0,NaN


In [100]:
doc_ids['FP'].value_counts()

1.0    17324
2.0     6614
3.0     1210
4.0      103
5.0        4
Name: FP, dtype: int64

Есть документы, которые часто ошибочно определяются как ложно-положительные. Можно создать список и посмотреть ассессеров, 
которые с ними работали

In [98]:
fp_docs = doc_ids[doc_ids['FP']>3].reset_index()['docid'].to_list()

In [104]:
df[df.docid.isin(fp_docs)].login.value_counts()

assessor3      6
assessor103    5
assessor157    5
assessor155    5
assessor20     4
              ..
assessor268    1
assessor27     1
assessor304    1
assessor599    1
assessor573    1
Name: login, Length: 344, dtype: int64

Можно сказать, что третьему ассессору не особо повезло с документами, но их было всего 6 из 206 ложно-положительных. Теория оказалось ошибочной, но проверить стоило

P.S. Можно посмотреть учебный проект на подобную тематику https://www.kaggle.com/viacheslavivanov99/sf-dst-11-credit-scoring-ivanov-viacheslav